In [4]:
import pandas as pd
import numpy as np
np.random.seed(0)

In [6]:
# lexicon_sentiments = pd.read_csv('product_targeted_sentiments.csv', index_col=0)    
# aspect_names = pd.read_csv('handmade_noun_fts.csv')
# # Map column names to the aspect names
# lexicon_sentiments.columns = aspect_names['noun']
# lexicon_sentiments.head()
# # save the lexicon
# lexicon_sentiments.to_csv('product_targeted_sentiments.csv')

In [53]:
def classify_sentiment(sentiment):
    ''' Classify sentiment as positive, negative, or neutral based on compound score. '''
    if sentiment >= 0.05:
        return 1 # positive
    elif sentiment <= -0.05:
        return -1 # negative
    else:
        return 0 # neutral
    
def classify_sentiment_binary(sentiment):
    ''' Classify sentiment as positive or negative based on compound score. '''
    if sentiment >= 0:
        return 1
    else:
        return -1

In [2]:
# Load predicted review-level targeted sentiments
review_targ_sentiments = pd.read_csv('review_targeted_sentiments.csv', index_col=0)
aspect_names = pd.read_csv('handmade_noun_fts.csv')
# Convert sentiment scores to sentiment classes
review_targ_sentiments['prediction'] = review_targ_sentiments['sentiment'].apply(classify_sentiment)
review_targ_sentiments.head()

,asin,review,gift,quality,product,order,picture,size,time,day,...,lettering,adult,purse,customize,help,amount,wait,soap,mess,door
0,B081TVVM16,boyfriend christmas love,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,B01CCWR3OO,extremely pleased purchase,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,B07PC4ZTGB,perfect gift golf lover,0.289683,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,B079T97FNX,get christmas gift wife love perfect kitchen u...,0.114668,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,B01DJJ6IQI,great product,NaN,NaN,0.229888,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
handmade_reviews_bal = pd.read_csv('handmade_reviews_balanced.csv').drop(columns=['title', 'asin']).dropna(subset=['text'])
handmade_reviews_orig = pd.read_csv('handmade_reviews.csv').drop(columns=['title', 'asin', 'user_id', 'helpful_vote', 'timestamp', 'verified_purchase', 'images']).dropna(subset=['text'])
print(len(handmade_reviews_bal))
print(len(handmade_reviews_orig))
# rename column parent_asin to asin
handmade_reviews_bal.rename(columns={'parent_asin': 'asin'}, inplace=True)
handmade_reviews_bal

26236
161915


,rating,text,asin
0,5.0,boyfriend christmas love,B081TVVM16
1,5.0,extremely pleased purchase,B01CCWR3OO
2,5.0,perfect gift golf lover,B07PC4ZTGB
3,5.0,get christmas gift wife love perfect kitchen u...,B079T97FNX
4,5.0,great product,B01DJJ6IQI
...,...,...,...
26330,2.0,charge product say 925the comfort awful due cl...,B08YK1H88V
26331,2.0,purchase coworker nice time inside include rec...,B0C9FQJS34
26332,2.0,disappoint capitalize first letter name overal...,B01FT98CB0
26333,2.0,cheaply make carbiner break within couple day ...,B083Q2N1KQ


In [70]:
# Sample 20 reviews for manual evaluation, from each star rating
test_reviews = handmade_reviews_bal.groupby('rating').apply(lambda x: x.sample(4), include_groups=False)
for rev in test_reviews['text']:
    # print review row number and rev
    print(rev)
# test_reviews.to_csv('handmade_reviews_balanced_test.csv', index=False)

receive yesterday chain pop cause piece fall money actual piece inscription cute something quality chain listen gut one
specify pair receive one one easily fall ear consider ripoff answer one email
bead center unmovable match center look crooked bummer
understand groom name must list first want sure bride name first write ask company question could sure bride name first respond put bride name blank groom groom name blank bride would assure would correctly carefully guess receive picture expensive version course personalize groom name first disappointed recourse leave also insist date would accept without first two 0s proceed include first two finished picture look weird wrong stuck give granddaughter husband apology beware prospective buyer might get expect hope
small anticipate really see name ingrave
much small appear photo
like print much however buy describe singer sewing co print instead get b clayton print
buy bracelet end april comfortable wear would come loose time morning fall

In [79]:
# Find test review in handmade_reviews_bal and add to end of overlap_df
test_overlap_df = pd.DataFrame(columns=handmade_reviews_bal.columns)
for rev in test_reviews['text']:
    test_overlap_df.loc[len(test_overlap_df)] = handmade_reviews_bal[handmade_reviews_bal['text'] == rev].iloc[0]

In [112]:
# Drop rating before displaying
test_overlap_df.drop(columns=['rating'])

,text,asin
0,receive yesterday chain pop cause piece fall m...,B07PGR2G13
1,specify pair receive one one easily fall ear c...,B0721V9WYH
2,bead center unmovable match center look crooke...,B08XXW5NCF
3,understand groom name must list first want sur...,B01DB9BFAK
4,small anticipate really see name ingrave,B01N5SVHUU
5,much small appear photo,B0C574VVJY
6,like print much however buy describe singer se...,B0BMRR2LS2
7,buy bracelet end april comfortable wear would ...,B09FT4MJRM
8,polish ok coat start chip 2nd day 4th day real...,B083QR47NK
9,grainy cute,B0BHJMLN2M


In [ ]:
# Manually searching for original reviews. replace target_asin with the asin to search for, then look through resulting review text to find matching
target_asin = 'B0BMB4QLZR'
matching_asins = handmade_reviews_orig[handmade_reviews_orig['parent_asin'] == target_asin]
for i, item in enumerate(matching_asins['text']):
    print(matching_asins['index'].iloc[i], item)

### ORIGINAL Reviews in test set
- Label the nouns/features with a positive, negative, or neutral sentiment. 

53428 I just received it yesterday and the chain popped. That caused the piece to fall out, so now I’m out of my money. The actual piece with the inscription was cute but they have to do something about the quality of the chain. I should have listened to me gut on this one.
- chain: 
21659 It specified a pair.  I received one.  The one easily falls off the ear.  I consider this a rip-off.  They have not answered one of my emails!

114453 The beads are off center and unmovable so it doesn’t match the center. Looks crooked. Bummer.

17196 I do not understand why the groom's name MUST be listed first, and wanted to be SURE the bride's name was first, so wrote and asked the company this question--how could I be sure the bride's name was first.  They responded that I should just put the bride's name in the blank for the groom, and the groom's name in the blank for the bride, which would assure that it would be done correctly.  I very carefully did this, but you guessed it- I just received the picture (the most expensive version, $160, of course) and it had been personalized with the groom's name first.  I'm so disappointed, but no recourse left to me.<br />Also, they INSISTED on 0's in the date or would not accept it without those first two 0s:  04-09-2019, and then proceeded to include the first two 0's on the finished picture.  Looks weird and just WRONG, but I'm stuck with it, and will just give it to my granddaughter and husband with apologies, which is not what I should have to do.<br /> Beware, any prospective buyer--you might not get what you were expecting and hoping for...

118630 Smaller than I anticipated and you can really see the name ingraved

153053 Much smaller than appeared on the photo

14910 I like the print very much. However, I bought it because it was described as SINGER SEWING Co. print. Instead I got an A. B. Clayton print.

102860 Bought this bracelet at the end of April. Comfortable to wear but would come loose at times. This morning (8/17/22) it just fell apart. I lost a few beads when it came apart. Really sucks because I really like this bracelet. Won't be buying another one since it won't last very long. Especially not at the current price.

130312 This polish is just OK. After 2 coats it started chipping the 2nd day. And on the 4th day it really doesn't look matte anymore.

67870 just grainy but it’s cute

2517 I should've paid more attention to the sizing because I ordered a medium for our cat which is WAY too big. Although I worry that a smaller size would be hard to read. Also I ordered the industrial steel and it's quite heavy. I'm not sure if the other materials would be lighter.<br />Unfortunately we won't be putting this on our cat and can't exactly return it. I did like the other trinkets that came with it.

153363 I love this pair and I didn't send them back, but I have tiny ears so they don't fit great. I wish there was more information about their size when buying them, or that there were options to get a pair that was shorter in length. If you don't have disproportionately tiny ears, they'll probably look great on you.

88344 Just as expected

80129 Although I haven’t applied them yet it looks pretty simple. I like this option better than painting my entire room. So far I’m pleased!

123655 Love the sign. I just wish it had a more sturdy hanging mechanism.

99411 Beautiful, but no where what I expected.  Not, worth the money.

83565 Amazing artwork! Bought this as a gift and it was very well received. Love that Amazon is promoting smaller businesses and artists!

157944 The ring is beautiful and goes with a lot of outfits. Very happy with my purchase

4719 Would order at least 2 more

107603 Search cane master circle of masters for compliance for air travel. Looks great, I would buy one if I didn't have my dad's.

In [34]:
test_revs_processed = ["receive yesterday chain pop cause piece fall money actual piece inscription cute something quality chain listen gut one",
"specify pair receive one one easily fall ear consider ripoff answer one email",
"bead center unmovable match center look crooked bummer",
"understand groom name must list first want sure bride name first write ask company question could sure bride name first respond put bride name blank groom groom name blank bride would assure would correctly carefully guess receive picture expensive version course personalize groom name first disappointed recourse leave also insist date would accept without first two 0s proceed include first two finished picture look weird wrong stuck give granddaughter husband apology beware prospective buyer might get expect hope",
"small anticipate really see name ingrave",
"much small appear photo",
"like print much however buy describe singer sewing co print instead get b clayton print",
"buy bracelet end april comfortable wear would come loose time morning fall apart lose bead come apart really suck really like bracelet buy another one since last long especially current price",
"polish ok coat start chip 2nd day 4th day really look matte anymore",
"grainy cute",
"pay attention sizing order medium cat way big although worry small size would hard read also order industrial steel quite heavy sure material would light unfortunately put cat exactly return like trinket come",
"love pair send back tiny ear fit great wish information size buy option get pair short length disproportionately tiny ear probably look great",
"expect",
"although apply yet look pretty simple like option well paint entire room far pleased",
"love sign wish sturdy hanging mechanism",
"beautiful expect worth money",
"amazing artwork buy gift well receive love amazon promote small business artist",
"ring beautiful go lot outfits happy purchase",
"would order least",
"search cane master circle master compliance air travel look great would buy one dad"]

In [ ]:
def find_aspect(aspect_name):
    ''' Returns the row in aspect_names that matches the aspect_name. Used to see if aspect_name is an aspect considered in targeted sentiment.'''
    return aspect_names[aspect_names['noun'] == aspect_name]

find_aspect('box')

Manually label each review's aspects above.

Compare with predicted targeted sentiments.

In [38]:
# # Get back overall ratings for test set reviews
# # Find test_revs_processed in handmade_reviews_bal
# test_revs_labeled = pd.DataFrame(columns=handmade_reviews_bal.columns)
# for rev in test_revs_processed:
#     test_revs_labeled.loc[len(test_revs_labeled)] = handmade_reviews_bal[handmade_reviews_bal['text'] == rev].iloc[0]
# # sort by rating
# test_revs_labeled.sort_values(by='rating', inplace=True)
# test_revs_labeled

,rating,text,asin
0,1.0,receive yesterday chain pop cause piece fall m...,B07PGR2G13
1,1.0,specify pair receive one one easily fall ear c...,B0721V9WYH
2,1.0,bead center unmovable match center look crooke...,B08XXW5NCF
3,1.0,understand groom name must list first want sur...,B01DB9BFAK
4,2.0,small anticipate really see name ingrave,B01N5SVHUU
5,2.0,much small appear photo,B0C574VVJY
6,2.0,like print much however buy describe singer se...,B0BMRR2LS2
7,2.0,buy bracelet end april comfortable wear would ...,B09FT4MJRM
11,3.0,love pair send back tiny ear fit great wish in...,B01NAYBY1S
10,3.0,pay attention sizing order medium cat way big ...,B09GK2JJDZ


## Whole sentiment Evaluation

In [59]:
whole_sent = pd.read_csv("whole_sentiment_output.csv")
original_review_df = pd.read_csv("handmade_reviews_balanced.csv").dropna(subset=['text']) #! need to drop na!
whole_sent['rating'] = original_review_df['rating'] # Get labels as ratings

# whole_sent['predicted'] = whole_sent['sentiment'].apply(classify_sentiment) # Convert sentiment scores to sentiment classes
# whole_sent['rating'] = whole_sent['rating'].apply(lambda x: 1 if x > 3 else (-1 if x < 3 else 0)) # Convert 5-star rating - positive, negative, neutral

whole_sent['predicted'] = whole_sent['sentiment'].apply(classify_sentiment_binary) # Convert sentiment scores to sentiment classes
whole_sent['label'] = whole_sent['rating'].apply(lambda x: 1 if x >= 3 else -1) # binary labels: positive, negative

whole_sent

,review,sentiment,predicted,label,rating
0,boyfriend christmas love,0.6369,1,1,5.0
1,extremely pleased purchase,0.4927,1,1,5.0
2,perfect gift golf lover,0.8860,1,1,5.0
3,get christmas gift wife love perfect kitchen u...,0.8957,1,1,5.0
4,great product,0.6249,1,1,5.0
...,...,...,...,...,...
26231,charge product say 925the comfort awful due cl...,-0.1065,-1,-1,2.0
26232,purchase coworker nice time inside include rec...,0.6115,1,-1,2.0
26233,disappoint capitalize first letter name overal...,0.3612,1,-1,2.0
26234,cheaply make carbiner break within couple day ...,0.5106,1,-1,2.0


In [52]:
# number of neutral predicted
whole_sent[whole_sent['predicted'] == 0]

,review,sentiment,predicted,label,rating
17,think come set board,0.0,0,1.0,1
23,work,0.0,0,1.0,1
197,excelente para mi boda,0.0,0,1.0,1
198,es un bonito artículo para regalar para tenerl...,0.0,0,1.0,1
280,linda fuerte,0.0,0,1.0,1
...,...,...,...,...,...
26217,clip bigheavy fall time pull hair slip,0.0,0,-1.0,-1
26220,come frame expensive get,0.0,0,-1.0,-1
26221,flimsy piece paper expect almost,0.0,0,-1.0,-1
26225,cheaply make,0.0,0,-1.0,-1


In [50]:
# Confusion matrix
def get_stats(predictions_df):
    TP = len(predictions_df[(predictions_df['predicted'] == 1) & (predictions_df['label'] == 1)])
    TN = len(predictions_df[(predictions_df['predicted'] == -1) & (predictions_df['label'] == -1)])
    FP = len(predictions_df[(predictions_df['predicted'] == 1) & (predictions_df['label'] == -1)])
    FN = len(predictions_df[(predictions_df['predicted'] == -1) & (predictions_df['label'] == 1)])
    print("TP: ", TP)
    print("TN: ", TN)
    print("FP: ", FP)
    print("FN: ", FN)
    print("Precision: ", TP / (TP + FP))
    print("Recall: ", TP / (TP + FN))
    print("F1 score: ", 2 * TP / (2 * TP + FP + FN))
    print("Accuracy: ", (TP + TN) / (TP + TN + FP + FN))

In [60]:
get_stats(whole_sent)

TP:  14874
TN:  2701
FP:  7793
FN:  868
Precision:  0.6561962324083469
Recall:  0.9448608817176979
F1 score:  0.7745059751620714
Accuracy:  0.6698810794328404
